# Cross-check .nc file differences between input directories

This is forked from https://github.com/COSIMA/input_om2-bgc/blob/25889e8d220d0c0fb0e48a617539fdd0bc22ebe5/check_updates.ipynb

Set `old`, `new`, `ofiles` and `nfiles` as appropriate.

If you run out of memory, try
```
gadi_jupyter -t 5:00:00 -m 1400GB -n 1 -P x77 -q hugemem
```
see https://opus.nci.org.au/display/Help/Queue+Limits

In [1]:
import dask.array as da
import xarray as xr
import numpy as np
import glob
import os.path
import warnings

In [2]:
import climtas.nci
climtas.nci.GadiClient(malloc_trim_threshold='64kib')

/g/data/hh5/public/apps/miniconda3/envs/analysis3-23.04/lib/python3.9/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 36309 instead
  warnings.warn(


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /proxy/36309/status,
Dashboard: /proxy/36309/status,Workers: 7
Total threads: 7,Total memory: 32.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:38721,Workers: 7
Dashboard: /proxy/36309/status,Total threads: 7
Started: Just now,Total memory: 32.00 GiB
Comm: tcp://127.0.0.1:42217,Total threads: 1
Dashboard: /proxy/43501/status,Memory: 4.57 GiB
Nanny: tcp://127.0.0.1:35895,


In [3]:
def checkvars():
    if ofiles != nfiles:
        print('ERROR: old and new file lists differ')
    else:
        for of, nf in zip(ofiles, nfiles):
            # print(of, nf)
            print(old+of)
            print(new+nf)
            print('ncview', old+of, new+nf, '&')
            print('     v, idcl, eql, np.nanmin(ratio), np.nanmax(ratio), np.nanmin(diff), np.nanmax(diff)')
            dsold = xr.open_dataset(old+of, decode_times=False, engine='netcdf4')
            dsnew = xr.open_dataset(new+nf, decode_times=False, engine='netcdf4')
            ovars = set(dsold.keys())
            nvars = set(dsnew.keys())
            if ovars != nvars:
                print('  NOTE: variable names differ - will only check the common variables')
                print('  ovars:', ovars)
                print('  nvars:', nvars)
            vs = list(ovars & nvars)
            vs.sort()
            for v in vs:
                # print([k for k in dsnew[v].coords.keys()])
                idcl = dsold[v].identical(dsnew[v])
                if not idcl:
                    eql = dsold[v].equals(dsnew[v])
                else:
                    eql = True  # identical implies equal
                if eql:
                    print('    ', v+repr([k for k in dsnew[v].coords.keys()]), idcl, eql)
                else:
                    try:
                        ratio = dsnew[v].data/dsold[v].data
                        diff = dsnew[v].data - dsold[v].data
                        print('    ', v+repr([k for k in dsnew[v].coords.keys()]), idcl, eql, np.nanmin(ratio), np.nanmax(ratio), np.nanmin(diff), np.nanmax(diff))
                    except:
                        print('    ', v+repr([k for k in dsnew[v].coords.keys()]), idcl, eql)

In [4]:
dirs = ['/scratch/v45/aek156/access-om3/archive/MOM6-CICE6_ACCESS-OM3_repro_test_1', 
        '/scratch/v45/aek156/access-om3/archive/MOM6-CICE6_ACCESS-OM3_repro_test_2',
       ]
dirtype = '/restart000/'
# dirtype = '/output000/'
old = dirs[0]+dirtype
new = dirs[1]+dirtype
print(old)
print(new)
ofiles = [ os.path.relpath(fpath, old) for fpath in glob.glob(old+'*.nc') ]
ofiles.sort()
nfiles = [ os.path.relpath(fpath, new) for fpath in glob.glob(new+'*.nc') ]
nfiles.sort()
# print('ofiles =', ofiles)
# print('nfiles =', nfiles)
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    checkvars()

/scratch/v45/aek156/access-om3/archive/MOM6-CICE6_ACCESS-OM3_repro_test_1/restart000/
/scratch/v45/aek156/access-om3/archive/MOM6-CICE6_ACCESS-OM3_repro_test_2/restart000/
/scratch/v45/aek156/access-om3/archive/MOM6-CICE6_ACCESS-OM3_repro_test_1/restart000/GMOM_JRA.cice.r.0001-01-02-00000.nc
/scratch/v45/aek156/access-om3/archive/MOM6-CICE6_ACCESS-OM3_repro_test_2/restart000/GMOM_JRA.cice.r.0001-01-02-00000.nc
ncview /scratch/v45/aek156/access-om3/archive/MOM6-CICE6_ACCESS-OM3_repro_test_1/restart000/GMOM_JRA.cice.r.0001-01-02-00000.nc /scratch/v45/aek156/access-om3/archive/MOM6-CICE6_ACCESS-OM3_repro_test_2/restart000/GMOM_JRA.cice.r.0001-01-02-00000.nc &
     v, idcl, eql, np.nanmin(ratio), np.nanmax(ratio), np.nanmin(diff), np.nanmax(diff)
     FY[] False False 0.0 inf -0.9999999999998583 1.0
     Tsfcn[] False False -0.0 5.7483917559558915 -14.396531551982138 11.86398023639946
     aeroiceint001[] False False 0.0 inf -3.3047145720394164e-10 7.736457344916413e-10
     aeroiceint002[

## old stuff below

In [10]:
dirs = [ '/scratch/v45/aek156/MOM6-CICE6_cesm', 
        '/scratch/v45/mo1833/access-om3/mo1833/cesm/archive/MOM6-CICE6',
        '/scratch/v45/mo1833/access-om3_master/mo1833/cesm/archive/MOM6-CICE6_master',
       ]
dirtype = '/restart000/'
dirtype = '/output000/'
old = dirs[1]+dirtype
new = dirs[2]+dirtype
print(old)
print(new)
ofiles = [ os.path.relpath(fpath, old) for fpath in glob.glob(old+'*.nc') ]
ofiles.sort()
nfiles = [ os.path.relpath(fpath, new) for fpath in glob.glob(new+'*.nc') ]
nfiles.sort()
# print('ofiles =', ofiles)
# print('nfiles =', nfiles)
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    checkvars()

/scratch/v45/mo1833/access-om3/mo1833/cesm/archive/MOM6-CICE6/output000/
/scratch/v45/mo1833/access-om3_master/mo1833/cesm/archive/MOM6-CICE6_master/output000/
/scratch/v45/mo1833/access-om3/mo1833/cesm/archive/MOM6-CICE6/output000/GMOM_JRA.cice.h.0001-01.nc
/scratch/v45/mo1833/access-om3_master/mo1833/cesm/archive/MOM6-CICE6_master/output000/GMOM_JRA.cice.h.0001-01.nc
ncview /scratch/v45/mo1833/access-om3/mo1833/cesm/archive/MOM6-CICE6/output000/GMOM_JRA.cice.h.0001-01.nc /scratch/v45/mo1833/access-om3_master/mo1833/cesm/archive/MOM6-CICE6_master/output000/GMOM_JRA.cice.h.0001-01.nc &
     v, idcl, eql, np.nanmin(ratio), np.nanmax(ratio), np.nanmin(diff), np.nanmax(diff)
     ANGLE['TLON', 'TLAT', 'ULON', 'ULAT'] True True
     ANGLET['TLON', 'TLAT', 'ULON', 'ULAT'] True True
     HTE['TLON', 'TLAT', 'ULON', 'ULAT'] True True
     HTN['TLON', 'TLAT', 'ULON', 'ULAT'] True True
     Tsfc['time', 'TLON', 'TLAT', 'ULON', 'ULAT'] False False 0.22259311 1.5140306 -2.9775977 3.689518
     VG

In [5]:
old = '/g/data/ik11/restarts/access-om2-01/01deg_jra55v140_iaf_cycle4/restart851/'
new = '/scratch/jk72/pw6046/access-om2/archive/01deg_jra55v140_iaf_cycle4_rerun_from_1983/restart851/'
ofiles = [ os.path.relpath(fpath, old) for fpath in glob.glob(old+'*/*.nc') if 'CHUCKABLE' not in fpath]
ofiles.sort()
nfiles = [ os.path.relpath(fpath, new) for fpath in glob.glob(new+'*/*.nc') if 'CHUCKABLE' not in fpath ]
nfiles.sort()
checkvars()

ice/grid.nc ice/grid.nc
     angle True True
     angleT True True
     clat_t True True
     clat_u True True
     clon_t True True
     clon_u True True
     hte True True
     htn True True
     tarea True True
     tlat True True
     tlon True True
     uarea True True
     ulat True True
     ulon True True
ice/i2o.nc ice/i2o.nc
     aice_io True True
     alg_io True True
     form_io True True
     htflx_io True True
     licefh_io True True
     licefw_io True True
     lwflx_io True True
     melt_io True True
     nit_io True True
     press_io True True
     qflux_io True True
     rain_io True True
     runof_io True True
     shflx_io True True
     snow_io True True
     stflx_io True True
     strsu_io True True
     strsv_io True True
     swflx_io True True
     wnd10_io True True
ice/iced.1984-01-01-00000.nc ice/iced.1984-01-01-00000.nc
     Tsfcn True True
     aicen True True
     algalN True True
     bgc_N_sk True True
     bgc_Nit_sk True True
     iceumask True

/jobfs/77045909.gadi-pbs/ipykernel_3910424/27200376.py:27: RuntimeWarning: invalid value encountered in divide
  ratio = dsnew[v].data/dsold[v].data


     eta_nonbouss False False 0.0 1.0 -31.796804862400908 10.217915226055378
     eta_t True True
     eta_t_bar True True
     eta_u True True
     forcing_u_bt True True
     forcing_v_bt True True
     patm_t True True
     pbot_u True True
     udrho True True
     vdrho True True
ocean/ocean_bih_friction.res.nc ocean/ocean_bih_friction.res.nc
     bih_viscosity True True
ocean/ocean_density.res.nc ocean/ocean_density.res.nc
     denominator_r True True
     drhodS True True
     drhodT True True
     drhodz_zt True True
     pressure_at_depth True True
     rho True True
     rho_salinity True True
ocean/ocean_frazil.res.nc ocean/ocean_frazil.res.nc
     frazil True True
ocean/ocean_pot_temp.res.nc ocean/ocean_pot_temp.res.nc
     pot_temp True True
ocean/ocean_sbc.res.nc ocean/ocean_sbc.res.nc
     alg_surf True True
     frazil True True
     n_surf True True
     s_surf True True
     sea_lev True True
     t_surf True True
     u_surf True True
     v_surf True True
ocean/ocea

In [6]:
old = '/g/data/ik11/restarts/access-om2-01/01deg_jra55v140_iaf_cycle4/restart839/'
new = '/scratch/jk72/pw6046/access-om2/archive/01deg_jra55v140_iaf_cycle4_rerun_from_1980/restart839/'
ofiles = [ os.path.relpath(fpath, old) for fpath in glob.glob(old+'*/*.nc') if 'CHUCKABLE' not in fpath]
ofiles.sort()
nfiles = [ os.path.relpath(fpath, new) for fpath in glob.glob(new+'*/*.nc') if 'CHUCKABLE' not in fpath ]
nfiles.sort()
checkvars()

ice/grid.nc ice/grid.nc
     angle True True
     angleT True True
     clat_t True True
     clat_u True True
     clon_t True True
     clon_u True True
     hte True True
     htn True True
     tarea True True
     tlat True True
     tlon True True
     uarea True True
     ulat True True
     ulon True True
ice/i2o.nc ice/i2o.nc
     aice_io True True
     alg_io True True
     form_io True True
     htflx_io True True
     licefh_io True True
     licefw_io True True
     lwflx_io True True
     melt_io True True
     nit_io True True
     press_io True True
     qflux_io True True
     rain_io True True
     runof_io True True
     shflx_io True True
     snow_io True True
     stflx_io True True
     strsu_io True True
     strsv_io True True
     swflx_io True True
     wnd10_io True True
ice/iced.1981-01-01-00000.nc ice/iced.1981-01-01-00000.nc
     Tsfcn True True
     aicen True True
     algalN True True
     bgc_N_sk True True
     bgc_Nit_sk True True
     iceumask True

In [3]:
def checkem():
    if ofiles != nfiles:
        print('ERROR: old and new file lists differ')
    else:
        for of, nf in zip(ofiles, nfiles):
            print(of, nf, end=' ')
            dsold = xr.open_dataset(old+of, decode_times=False, engine='netcdf4')
            dsnew = xr.open_dataset(new+nf, decode_times=False, engine='netcdf4')
            if dsold.identical(dsnew):
                print(True, True)
            else:
                if dsold.equals(dsnew):
                    print(False, True)
                else:
                    print(False, False)
                    ovars = set(dsold.keys())
                    nvars = set(dsnew.keys())
                    if ovars != nvars:
                        print('  NOTE: variable names differ - will only check the common variables')
                        print('  ovars:', ovars)
                        print('  nvars:', nvars)
                    vs = list(ovars & nvars)
                    vs.sort()
                    for v in vs:
                        idcl = dsold[v].identical(dsnew[v])
                        if not idcl:
                            eql = dsold[v].equals(dsnew[v])
                        else:
                            eql = True  # identical implies equal
                        if eql:
                            print('    ', v, idcl, eql)
                        else:
                            try:
                                ratio = dsnew[v].data/dsold[v].data
                                diff = dsnew[v].data - dsold[v].data
                                print('    ', v, idcl, eql, np.nanmin(ratio), np.nanmax(ratio), np.nanmin(diff), np.nanmax(diff))
                            except:
                                print('    ', v, idcl, eql)

In [ ]:
old = '/g/data/ik11/restarts/access-om2-01/01deg_jra55v140_iaf_cycle4/restart851/'
new = '/scratch/jk72/pw6046/access-om2/archive/01deg_jra55v140_iaf_cycle4_rerun_from_1983/restart851/'
ofiles = [ os.path.relpath(fpath, old) for fpath in glob.glob(old+'*/*.nc') if 'CHUCKABLE' not in fpath]
ofiles.sort()
nfiles = [ os.path.relpath(fpath, new) for fpath in glob.glob(new+'*/*.nc') if 'CHUCKABLE' not in fpath ]
nfiles.sort()
checkem()

ice/grid.nc ice/grid.nc True True
ice/i2o.nc ice/i2o.nc True True
ice/iced.1984-01-01-00000.nc ice/iced.1984-01-01-00000.nc True True
ice/kmt.nc ice/kmt.nc True True
ice/monthly_sstsss.nc ice/monthly_sstsss.nc True True
ice/o2i.nc ice/o2i.nc True True
ice/sicemass.nc ice/sicemass.nc True True
ice/u_star.nc ice/u_star.nc True True
ocean/csiro_bgc.res.nc ocean/csiro_bgc.res.nc 

In [4]:
old = '/g/data/ik11/restarts/access-om2-01/01deg_jra55v140_iaf_cycle4/restart927/'
new = '/scratch/v45/aek156/access-om2/archive/01deg_jra55v140_iaf_cycle4_rerun_from_2002/restart927/'
ofiles = [ os.path.relpath(fpath, old) for fpath in glob.glob(old+'*/*.nc') if 'CHUCKABLE' not in fpath]
ofiles.sort()
nfiles = [ os.path.relpath(fpath, new) for fpath in glob.glob(new+'*/*.nc') if 'CHUCKABLE' not in fpath ]
nfiles.sort()
checkem()

ice/grid.nc ice/grid.nc True True
ice/i2o.nc ice/i2o.nc True True
ice/iced.2003-01-01-00000.nc ice/iced.2003-01-01-00000.nc True True
ice/kmt.nc ice/kmt.nc True True
ice/monthly_sstsss.nc ice/monthly_sstsss.nc True True
ice/o2i.nc ice/o2i.nc True True
ice/sicemass.nc ice/sicemass.nc True True
ice/u_star.nc ice/u_star.nc True True
ocean/csiro_bgc.res.nc ocean/csiro_bgc.res.nc False True
ocean/csiro_bgc_sediment.res.nc ocean/csiro_bgc_sediment.res.nc False True
ocean/ocean_age.res.nc ocean/ocean_age.res.nc False True
ocean/ocean_barotropic.res.nc ocean/ocean_barotropic.res.nc False False
     anompb True True
     anompb_bar True True
     conv_rho_ud_t True True


/jobfs/44196302.gadi-pbs/ipykernel_2910992/251776343.py:34: RuntimeWarning: invalid value encountered in true_divide
  ratio = dsnew[v].data/dsold[v].data


     eta_nonbouss False False -0.0 1.0 -34.54282011402216 11.28266317892498
     eta_t True True
     eta_t_bar True True
     eta_u True True
     forcing_u_bt True True
     forcing_v_bt True True
     patm_t True True
     pbot_u True True
     udrho True True
     vdrho True True
ocean/ocean_bih_friction.res.nc ocean/ocean_bih_friction.res.nc False True
ocean/ocean_density.res.nc ocean/ocean_density.res.nc False True
ocean/ocean_frazil.res.nc ocean/ocean_frazil.res.nc False True
ocean/ocean_pot_temp.res.nc ocean/ocean_pot_temp.res.nc False True
ocean/ocean_sbc.res.nc ocean/ocean_sbc.res.nc False True
ocean/ocean_temp_salt.res.nc ocean/ocean_temp_salt.res.nc False True
ocean/ocean_thickness.res.nc ocean/ocean_thickness.res.nc False True
ocean/ocean_velocity.res.nc ocean/ocean_velocity.res.nc False True
ocean/ocean_velocity_advection.res.nc ocean/ocean_velocity_advection.res.nc False True


In [4]:
old = '/g/data/ik11/inputs/access-om2/input_bgc_20211126/'
new = '/g/data/ik11/inputs/access-om2/input_bgc_20220224/'
ofiles = [ os.path.relpath(fpath, old) for fpath in glob.glob(old+'*/*.nc') if 'CHUCKABLE' not in fpath]
ofiles.sort()
nfiles = [ os.path.relpath(fpath, new) for fpath in glob.glob(new+'*/*.nc') if 'CHUCKABLE' not in fpath ]
nfiles.sort()
checkem()

In [5]:
old = '/g/data/ik11/inputs/access-om2/input_bgc_20220224/mom_01deg/'
new = '/g/data/ik11/inputs/access-om2/input_bgc_20220224/mom_01deg-cycle4/'
ofiles = ['csiro_bgc.res.nc']
nfiles = ['csiro_bgc.res.nc']
checkem()

In [12]:
old = '/g/data/ik11/inputs/access-om2/input_bgc_20220224/mom_01deg-cycle4/'
new = '/scratch/v45/aek156/access-om2/archive/01deg_jra55v140_iaf_cycle4/restart831/ocean/'
ofiles = ['csiro_bgc.res.nc']
nfiles = ['csiro_bgc.res.nc']
checkem()

csiro_bgc.res.nc csiro_bgc.res.nc False False
     adic False False 0.0 2.476517944427492e+34 -2573.983539968264 9.969209968386869e+36
     alk False False 0.0 2.5712322550726645e+34 -2721.7085608856787 9.969209968386869e+36
     caco3 False False 0.0 9.969209968386869e+38 -0.01 9.969209968386869e+36
     det False False 0.0 9.969209968386869e+38 -0.01 9.969209968386869e+36
     dic False False 0.0 2.730771899743082e+34 -2574.3003062674875 9.969209968386869e+36
     fe False False 0.0 6.293195507175558e+39 -3.5727932113904166 9.969209968386869e+36


/jobfs/36180039.gadi-pbs/ipykernel_773822/1466042173.py:34: RuntimeWarning: divide by zero encountered in true_divide
  ratio = dsnew[v].data/dsold[v].data
/jobfs/36180039.gadi-pbs/ipykernel_773822/1466042173.py:34: RuntimeWarning: invalid value encountered in true_divide
  ratio = dsnew[v].data/dsold[v].data


     no3 False False 0.0 inf -46.895320034262035 9.969209968386869e+36
     o2 False False 1.0 1.0 0.0 0.0
     phy False False 0.0 9.969209968386869e+38 -0.01 9.969209968386869e+36
     zoo False False 0.0 9.969209968386869e+38 -0.01 9.969209968386869e+36


In [11]:
old = '/scratch/v45/hh0162/access-om2/archive/01deg_jra55v140_iaf_cycle4/restart831/ocean/'
new = '/scratch/v45/aek156/access-om2/archive/01deg_jra55v140_iaf_cycle4/restart831/ocean/'
ofiles = ['csiro_bgc.res.nc']
nfiles = ['csiro_bgc.res.nc']
checkem()

csiro_bgc.res.nc csiro_bgc.res.nc False False
     adic True True
     alk True True
     caco3 True True
     det True True
     dic True True
     fe True True
     no3 True True


/jobfs/36180039.gadi-pbs/ipykernel_773822/1466042173.py:34: RuntimeWarning: divide by zero encountered in true_divide
  ratio = dsnew[v].data/dsold[v].data


     o2 False False 0.0 inf -9.969209968386869e+36 463.5191535084184
     phy True True
     zoo True True


In [7]:
old = '/g/data/ik11/outputs/access-om2-01/01deg_jra55v140_iaf_cycle4/output833/ocean/'
new = '/scratch/v45/aek156/access-om2/archive/01deg_jra55v140_iaf_cycle4/output833/ocean/'
ofiles = [ 'oceanbgc-2d-surface_'+n+'-1-daily-mean-ym_1979_04.nc' for n in ['adic', 'alk', 'caco3', 'det', 'dic', 'fe', 'no3', 'o2', 'phy', 'zoo'] ]
nfiles = ofiles
checkem()

oceanbgc-2d-surface_adic-1-daily-mean-ym_1979_04.nc oceanbgc-2d-surface_adic-1-daily-mean-ym_1979_04.nc False True
oceanbgc-2d-surface_alk-1-daily-mean-ym_1979_04.nc oceanbgc-2d-surface_alk-1-daily-mean-ym_1979_04.nc False True
oceanbgc-2d-surface_caco3-1-daily-mean-ym_1979_04.nc oceanbgc-2d-surface_caco3-1-daily-mean-ym_1979_04.nc False True
oceanbgc-2d-surface_det-1-daily-mean-ym_1979_04.nc oceanbgc-2d-surface_det-1-daily-mean-ym_1979_04.nc False True
oceanbgc-2d-surface_dic-1-daily-mean-ym_1979_04.nc oceanbgc-2d-surface_dic-1-daily-mean-ym_1979_04.nc False True
oceanbgc-2d-surface_fe-1-daily-mean-ym_1979_04.nc oceanbgc-2d-surface_fe-1-daily-mean-ym_1979_04.nc False True
oceanbgc-2d-surface_no3-1-daily-mean-ym_1979_04.nc oceanbgc-2d-surface_no3-1-daily-mean-ym_1979_04.nc False True
oceanbgc-2d-surface_o2-1-daily-mean-ym_1979_04.nc oceanbgc-2d-surface_o2-1-daily-mean-ym_1979_04.nc False False
     average_DT True True
     average_T1 True True
     average_T2 True True
     surface_o2

In [8]:
old = '/g/data/ik11/outputs/access-om2-01/01deg_jra55v140_iaf_cycle4/output833/ocean/'
new = '/scratch/v45/aek156/access-om2/archive/01deg_jra55v140_iaf_cycle4/output833/ocean/'
ofiles = [ 'ocean-2d-'+n+'-mean-ym_1979_04.nc' for n in ['surface_salt-1-daily', 'surface_pot_temp-1-daily', 'usurf-1-daily', 'vsurf-1-daily'] ]
nfiles = ofiles
checkem()

ocean-2d-surface_salt-1-daily-mean-ym_1979_04.nc ocean-2d-surface_salt-1-daily-mean-ym_1979_04.nc False True
ocean-2d-surface_pot_temp-1-daily-mean-ym_1979_04.nc ocean-2d-surface_pot_temp-1-daily-mean-ym_1979_04.nc False True
ocean-2d-usurf-1-daily-mean-ym_1979_04.nc ocean-2d-usurf-1-daily-mean-ym_1979_04.nc False True
ocean-2d-vsurf-1-daily-mean-ym_1979_04.nc ocean-2d-vsurf-1-daily-mean-ym_1979_04.nc False True


In [6]:
old = '/scratch/v45/hh0162/access-om2/archive/01deg_jra55v140_iaf_cycle4/restart835/'
new = '/scratch/v45/aek156/access-om2/archive/01deg_jra55v140_iaf_cycle4/restart835/'
ofiles = [ os.path.relpath(fpath, old) for fpath in glob.glob(old+'*/*.nc') if 'CHUCKABLE' not in fpath]
ofiles.sort()
nfiles = [ os.path.relpath(fpath, new) for fpath in glob.glob(new+'*/*.nc') if 'CHUCKABLE' not in fpath ]
nfiles.sort()
checkem()

ice/grid.nc ice/grid.nc True True
ice/i2o.nc ice/i2o.nc True True
ice/iced.1980-01-01-00000.nc ice/iced.1980-01-01-00000.nc True True
ice/kmt.nc ice/kmt.nc True True
ice/monthly_sstsss.nc ice/monthly_sstsss.nc True True
ice/o2i.nc ice/o2i.nc True True
ice/sicemass.nc ice/sicemass.nc True True
ice/u_star.nc ice/u_star.nc True True
ocean/csiro_bgc.res.nc ocean/csiro_bgc.res.nc False False
     adic True True
     alk True True
     caco3 True True
     det True True
     dic True True
     fe True True
     no3 True True


/jobfs/36387466.gadi-pbs/ipykernel_70696/1466042173.py:34: RuntimeWarning: divide by zero encountered in true_divide
  ratio = dsnew[v].data/dsold[v].data
/jobfs/36387466.gadi-pbs/ipykernel_70696/1466042173.py:34: RuntimeWarning: invalid value encountered in true_divide
  ratio = dsnew[v].data/dsold[v].data


     o2 False False 0.0 inf -13121.305162308845 362.71752554986796
     phy True True
     zoo True True
ocean/csiro_bgc_sediment.res.nc ocean/csiro_bgc_sediment.res.nc False True
ocean/ocean_age.res.nc ocean/ocean_age.res.nc False True
ocean/ocean_barotropic.res.nc ocean/ocean_barotropic.res.nc False True
ocean/ocean_bih_friction.res.nc ocean/ocean_bih_friction.res.nc False True
ocean/ocean_density.res.nc ocean/ocean_density.res.nc False True
ocean/ocean_frazil.res.nc ocean/ocean_frazil.res.nc False True
ocean/ocean_pot_temp.res.nc ocean/ocean_pot_temp.res.nc False True
ocean/ocean_sbc.res.nc ocean/ocean_sbc.res.nc False True
ocean/ocean_temp_salt.res.nc ocean/ocean_temp_salt.res.nc False True
ocean/ocean_thickness.res.nc ocean/ocean_thickness.res.nc False True
ocean/ocean_velocity.res.nc ocean/ocean_velocity.res.nc False True
ocean/ocean_velocity_advection.res.nc ocean/ocean_velocity_advection.res.nc False True


In [9]:
old = '/g/data/ik11/outputs/access-om2-01/01deg_jra55v140_iaf_cycle4_OLD/output834/ocean/'
new = old
ofiles = ['oceanbgc-3d-fe-1-monthly-mean-ym_1979_09.nc-orig']
nfiles = ['oceanbgc-3d-fe-1-monthly-mean-ym_1979_09.nc']
checkem()

oceanbgc-3d-fe-1-monthly-mean-ym_1979_09.nc-orig oceanbgc-3d-fe-1-monthly-mean-ym_1979_09.nc False True


In [19]:
old = '/g/data/ik11/outputs/access-om2-01/01deg_jra55v140_iaf_cycle4_OLD/CHUCKABLE-output882/ocean/'
new = '/g/data/ik11/outputs/access-om2-01/01deg_jra55v140_iaf_cycle4_OLD/output882/ocean/'
ofiles = [ os.path.relpath(fpath, old) for fpath in glob.glob(old+'oceanbgc-2d-*_int*.nc')]
ofiles.sort()
nfiles = [ os.path.relpath(fpath, new) for fpath in glob.glob(new+'oceanbgc-2d-*_int*.nc')]
nfiles.sort()
checkem()

oceanbgc-2d-det_int100-1-daily-mean-ym_1991_07.nc oceanbgc-2d-det_int100-1-daily-mean-ym_1991_07.nc False True
oceanbgc-2d-det_int100-1-daily-mean-ym_1991_08.nc oceanbgc-2d-det_int100-1-daily-mean-ym_1991_08.nc False True
oceanbgc-2d-det_int100-1-daily-mean-ym_1991_09.nc oceanbgc-2d-det_int100-1-daily-mean-ym_1991_09.nc False True
oceanbgc-2d-det_intmld-1-daily-mean-ym_1991_07.nc oceanbgc-2d-det_intmld-1-daily-mean-ym_1991_07.nc False True
oceanbgc-2d-det_intmld-1-daily-mean-ym_1991_08.nc oceanbgc-2d-det_intmld-1-daily-mean-ym_1991_08.nc False True
oceanbgc-2d-det_intmld-1-daily-mean-ym_1991_09.nc oceanbgc-2d-det_intmld-1-daily-mean-ym_1991_09.nc False True
oceanbgc-2d-fe_int100-1-daily-mean-ym_1991_07.nc oceanbgc-2d-fe_int100-1-daily-mean-ym_1991_07.nc False True
oceanbgc-2d-fe_int100-1-daily-mean-ym_1991_08.nc oceanbgc-2d-fe_int100-1-daily-mean-ym_1991_08.nc False True
oceanbgc-2d-fe_int100-1-daily-mean-ym_1991_09.nc oceanbgc-2d-fe_int100-1-daily-mean-ym_1991_09.nc False True
oceanbg

In [6]:
old = '/g/data/ik11/restarts/access-om2-01/01deg_jra55v140_iaf_cycle3/'
new = '/g/data/ik11/outputs/access-om2-01/01deg_jra55v140_iaf_cycle3/'
ofiles = [ os.path.relpath(fpath, old) for fpath in glob.glob(old+'restart*/*/*.nc')]
ofiles.sort()
nfiles = [ os.path.relpath(fpath, new) for fpath in glob.glob(new+'restart*/*/*.nc')]
nfiles.sort()

In [7]:
ofiles

['restart487/ice/grid.nc',
 'restart487/ice/i2o.nc',
 'restart487/ice/iced.2019-01-01-00000.nc',
 'restart487/ice/kmt.nc',
 'restart487/ice/monthly_sstsss.nc',
 'restart487/ice/o2i.nc',
 'restart487/ice/sicemass.nc',
 'restart487/ice/u_star.nc',
 'restart487/ocean/ocean_age.res.nc',
 'restart487/ocean/ocean_barotropic.res.nc',
 'restart487/ocean/ocean_bih_friction.res.nc',
 'restart487/ocean/ocean_density.res.nc',
 'restart487/ocean/ocean_frazil.res.nc',
 'restart487/ocean/ocean_pot_temp.res.nc',
 'restart487/ocean/ocean_sbc.res.nc',
 'restart487/ocean/ocean_temp_salt.res.nc',
 'restart487/ocean/ocean_thickness.res.nc',
 'restart487/ocean/ocean_velocity.res.nc',
 'restart487/ocean/ocean_velocity_advection.res.nc',
 'restart488/ice/grid.nc',
 'restart488/ice/i2o.nc',
 'restart488/ice/iced.1958-04-01-00000.nc',
 'restart488/ice/kmt.nc',
 'restart488/ice/monthly_sstsss.nc',
 'restart488/ice/o2i.nc',
 'restart488/ice/sicemass.nc',
 'restart488/ice/u_star.nc',
 'restart488/ocean/ocean_age.r

In [ ]:
checkem()

restart487/ice/grid.nc restart487/ice/grid.nc True True
restart487/ice/i2o.nc restart487/ice/i2o.nc True True
restart487/ice/iced.2019-01-01-00000.nc restart487/ice/iced.2019-01-01-00000.nc True True
restart487/ice/kmt.nc restart487/ice/kmt.nc True True
restart487/ice/monthly_sstsss.nc restart487/ice/monthly_sstsss.nc True True
restart487/ice/o2i.nc restart487/ice/o2i.nc True True
restart487/ice/sicemass.nc restart487/ice/sicemass.nc True True
restart487/ice/u_star.nc restart487/ice/u_star.nc True True
restart487/ocean/ocean_age.res.nc restart487/ocean/ocean_age.res.nc True True
restart487/ocean/ocean_barotropic.res.nc restart487/ocean/ocean_barotropic.res.nc True True
restart487/ocean/ocean_bih_friction.res.nc restart487/ocean/ocean_bih_friction.res.nc True True
restart487/ocean/ocean_density.res.nc restart487/ocean/ocean_density.res.nc 

In [8]:
nfiles

['restart487/ice/grid.nc',
 'restart487/ice/i2o.nc',
 'restart487/ice/iced.2019-01-01-00000.nc',
 'restart487/ice/kmt.nc',
 'restart487/ice/monthly_sstsss.nc',
 'restart487/ice/o2i.nc',
 'restart487/ice/sicemass.nc',
 'restart487/ice/u_star.nc',
 'restart487/ocean/ocean_age.res.nc',
 'restart487/ocean/ocean_barotropic.res.nc',
 'restart487/ocean/ocean_bih_friction.res.nc',
 'restart487/ocean/ocean_density.res.nc',
 'restart487/ocean/ocean_frazil.res.nc',
 'restart487/ocean/ocean_pot_temp.res.nc',
 'restart487/ocean/ocean_sbc.res.nc',
 'restart487/ocean/ocean_temp_salt.res.nc',
 'restart487/ocean/ocean_thickness.res.nc',
 'restart487/ocean/ocean_velocity.res.nc',
 'restart487/ocean/ocean_velocity_advection.res.nc',
 'restart488/ice/grid.nc',
 'restart488/ice/i2o.nc',
 'restart488/ice/iced.1958-04-01-00000.nc',
 'restart488/ice/kmt.nc',
 'restart488/ice/monthly_sstsss.nc',
 'restart488/ice/o2i.nc',
 'restart488/ice/sicemass.nc',
 'restart488/ice/u_star.nc',
 'restart488/ocean/ocean_age.r

In [5]:
xr.open_dataset('/scratch/v45/hh0162/access-om2/archive/01deg_jra55v140_iaf_cycle4/restart831/ocean/csiro_bgc.res.nc')['adic']

<xarray.DataArray 'adic' (Time: 1, zaxis_1: 75, yaxis_1: 2700, xaxis_1: 3600)>
[729000000 values with dtype=float64]
Coordinates:
  * xaxis_1  (xaxis_1) float64 1.0 2.0 3.0 4.0 ... 3.598e+03 3.599e+03 3.6e+03
  * yaxis_1  (yaxis_1) float64 1.0 2.0 3.0 4.0 ... 2.698e+03 2.699e+03 2.7e+03
  * zaxis_1  (zaxis_1) float64 1.0 2.0 3.0 4.0 5.0 ... 71.0 72.0 73.0 74.0 75.0
  * Time     (Time) float64 1.0
Attributes:
    long_name:  adic
    units:      none
    checksum:   EBBFDBD4EA81BECA

In [6]:
xr.open_dataset('/scratch/v45/aek156/access-om2/archive/01deg_jra55v140_iaf_cycle4/restart831/ocean/csiro_bgc.res.nc-original')['adic']

<xarray.DataArray 'adic' (Time: 1, zaxis_1: 75, yaxis_1: 2700, xaxis_1: 3600)>
[729000000 values with dtype=float64]
Coordinates:
  * xaxis_1  (xaxis_1) float64 1.0 2.0 3.0 4.0 ... 3.598e+03 3.599e+03 3.6e+03
  * yaxis_1  (yaxis_1) float64 1.0 2.0 3.0 4.0 ... 2.698e+03 2.699e+03 2.7e+03
  * zaxis_1  (zaxis_1) float64 1.0 2.0 3.0 4.0 5.0 ... 71.0 72.0 73.0 74.0 75.0
  * Time     (Time) float64 1.0
Attributes:
    long_name:  adic
    units:      none
    checksum:   EBBFDBD4EA81BECA

In [10]:
xr.open_dataset('/scratch/v45/aek156/access-om2/archive/01deg_jra55v140_iaf_cycle4/restart831/ocean/csiro_bgc.res.nc')['adic']

<xarray.DataArray 'adic' (Time: 1, zaxis_1: 75, yaxis_1: 2700, xaxis_1: 3600)>
[729000000 values with dtype=float64]
Coordinates:
  * xaxis_1  (xaxis_1) float64 1.0 2.0 3.0 4.0 ... 3.598e+03 3.599e+03 3.6e+03
  * yaxis_1  (yaxis_1) float64 1.0 2.0 3.0 4.0 ... 2.698e+03 2.699e+03 2.7e+03
  * zaxis_1  (zaxis_1) float64 1.0 2.0 3.0 4.0 5.0 ... 71.0 72.0 73.0 74.0 75.0
  * Time     (Time) float64 1.0
Attributes:
    long_name:  adic
    units:      none
    checksum:   EBBFDBD4EA81BECA

In [9]:
xr.open_dataset('/scratch/v45/aek156/access-om2/archive/01deg_jra55v140_iaf_cycle4/restart831/ocean/csiro_bgc.res.nc-attempt1')['adic']

<xarray.DataArray 'adic' (Time: 1, zaxis_1: 75, yaxis_1: 2700, xaxis_1: 3600)>
[729000000 values with dtype=float64]
Coordinates:
  * xaxis_1  (xaxis_1) float64 -279.9 -279.8 -279.7 -279.6 ... 79.75 79.85 79.95
  * yaxis_1  (yaxis_1) float64 -81.11 -81.07 -81.02 -80.98 ... 89.89 89.94 89.98
  * zaxis_1  (zaxis_1) float64 0.5413 1.681 2.94 ... 5.511e+03 5.709e+03
  * Time     (Time) object 0001-01-02 00:00:00
Attributes:
    long_name:  adic
    units:      none
    checksum:   EBBFDBD4EA81BECA

In [8]:
xr.open_dataset('/g/data/ik11/inputs/access-om2/input_bgc_20220224/mom_01deg-cycle4/csiro_bgc.res.nc')['adic']

/g/data/hh5/public/apps/miniconda3/envs/analysis3-21.10/lib/python3.9/site-packages/xarray/coding/times.py:526: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  dtype = _decode_cf_datetime_dtype(data, units, calendar, self.use_cftime)
/g/data/hh5/public/apps/miniconda3/envs/analysis3-21.10/lib/python3.9/site-packages/xarray/core/indexing.py:422: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  return np.asarray(array[self.key], dtype=None)


<xarray.DataArray 'adic' (Time: 1, zaxis_1: 75, yaxis_1: 2700, xaxis_1: 3600)>
[729000000 values with dtype=float64]
Coordinates:
  * xaxis_1  (xaxis_1) float64 -279.9 -279.8 -279.7 -279.6 ... 79.75 79.85 79.95
  * yaxis_1  (yaxis_1) float64 -81.11 -81.07 -81.02 -80.98 ... 89.89 89.94 89.98
  * zaxis_1  (zaxis_1) float64 0.5413 1.681 2.94 ... 5.511e+03 5.709e+03
  * Time     (Time) object 0001-01-02 00:00:00
Attributes:
    long_name:  natural + anthropogenic dissolved inorganic carbon
    units:      mmol/m^3